In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import scipy.signal
import time
import statistics

herz = 5000
def diff2(enter):
    d1=enter[1:]-enter[:-1]
    d2=d1[1:]-d1[:-1]
    return d2*d2

def SRfinder(qr,qs,mint, hint,data, begining_of_data_example, length_of_data_example, truncator, between_peak_min_distance_in_s, percent, treshold, prom, lenght_of_plot):
    
    chanels=data.columns.values[3:]
    timecolumn=data.columns.values[0]
    
    list_of_results1=pd.DataFrame()
    list_of_results2=pd.DataFrame()
    list_of_results3=pd.DataFrame()
    
    do_i_need_to_ask=False
    mintime=mint*herz
    if hint!=0:
      maxtime=hint*herz
    if hint==0:
        maxtime=len(data[timecolumn])
    
    
    
    
        
    for chanel_name in chanels:
        print("Chanel "+chanel_name+" is in progress.")
        x=data[timecolumn].values.astype("float")
        y=data[chanel_name].values.astype("float")*1000000000
        print(mintime)
        print(maxtime)
        print(len(x))
        x=x[mintime:maxtime-1]
        y=y[mintime:maxtime-1]
        plt.plot(x,y)
        plt.show()
        if do_i_need_to_ask:
            exx=x[begining_of_data_example:begining_of_data_example+length_of_data_example]
            exy=y[begining_of_data_example:begining_of_data_example+length_of_data_example]
            
            fig, axs = plt.subplots(2, figsize=(14,8))

            axs[0].plot(exx, exy, 'k')
            axs[1].plot(x, y, 'k')
            plt.show()
            time.sleep(1)
            print("Process this data? No if no")
            time.sleep(1)
            ans=input()
            if ans == 'no' or ans == 'No':
                print("Chanel "+chanel_name+" is skiped. Do you want continue with other chanels? Yes or No")
                time.sleep(1)
                ans=input()
                if ans == 'yes' or ans == '':
                    continue
                else:
                    if len(list_of_results1)>0:
                        return list_of_results1, list_of_results2, list_of_results3
                    else:
                        return None
        
        b_filter=False
        
        print("filter")

        if b_filter:
            b, a = scipy.signal.bessel(3, 0.005)
            filtered = scipy.signal.filtfilt(b, a, y)
            y=filtered

        
        y=np.round(np.real(y)*1000)/1000
        y=-y
        h=max(y)-min(y)
        lenx=len(x)
        b_continue=False
        
        
        while(True):
            
            tr=treshold
            pr=prom
            S_peak_index=scipy.signal.find_peaks(-y, distance=herz*between_peak_min_distance_in_s, height=-tr, prominence=pr)[0]
            S_peak_time=[]
            S_peak_value=[]
            for i in S_peak_index:
                S_peak_time.append(x[i])
                S_peak_value.append(y[i])
            
            
            if do_i_need_to_ask:
                number_of_plots=int(np.floor(lenx*3/lenght_of_plot))+1

                plotsx=np.array_split(x, number_of_plots)
                plotsy=np.array_split(y, number_of_plots)

                last_dot_index=0
                for i in range(number_of_plots):
                    fig = plt.figure( figsize=(14,8))
                    plt.plot(plotsx[i],plotsy[i], 'k')
                    plt.plot(plotsx[i],tr*np.ones(len(plotsx[i])), 'r')

                    number_of_dots=sum(np.logical_and(S_peak_time<=plotsx[i][-1], S_peak_time>plotsx[i][0]))

                    tempxS=S_peak_time[last_dot_index:last_dot_index+number_of_dots]
                    tempyS=S_peak_value[last_dot_index:last_dot_index+number_of_dots]
                    last_dot_index+=number_of_dots
                    plt.scatter(tempxS, tempyS)

                    plt.show()

                time.sleep(1)
                print("Does dots looks fine? No if no")
                time.sleep(1)
                ans=input()
                if ans == 'no' or ans == 'No':
                    print("You can change the percent value, by default it's equals to 0, \nif you increase it you can move the treshold down. Do you want it? Yes or no")
                    time.sleep(1)
                    ans=input()
                    if ans == 'yes' or ans == 'Yes':
                        print("Enter new value")
                        time.sleep(1)
                        percent=float(input())
                    else:
                        print("Chanel "+chanel_name+" is skiped. Do you want continue with other chanels? Yes or No")
                        time.sleep(1)
                        ans=input()
                        if ans == 'yes' or ans == 'Yes':
                            b_continue=True
                            break
                        else:
                            if len(list_of_results1)>0:
                                return list_of_results1, list_of_results2, list_of_results3
                            else:
                                return None
                else:
                    break
            else:
                break
        
        if b_continue:
            continue
        
        
        print("all_dots")
        R_peak_time=[]
        R_peak_value=[]
        par=1200
        Q_shift=qs
        Q_region_width=qr
        
        for i in S_peak_index:
            Rpeakindex=max(0,i+Q_shift-Q_region_width) + y[max(0,i+Q_shift-Q_region_width):min(i+Q_shift+Q_region_width, lenx)].argmax()
            R_peak_time.append(x[Rpeakindex])
            R_peak_value.append(y[Rpeakindex])
        
        Rarray=np.array(R_peak_value)
        Sarray=np.array(S_peak_value)
        Peak_coordinate_array=np.array(S_peak_time)
        
       
        number_of_plots=int(np.floor(lenx*3/lenght_of_plot))+1
        
        plotsx=np.array_split(x, number_of_plots)
        plotsy=np.array_split(y, number_of_plots)
        
        print("plots")
        last_dot_index=0
        
        for i in range(number_of_plots):
            fig = plt.figure( figsize=(14,8))
            plt.plot(plotsx[i],plotsy[i], 'k')
            
            number_of_dots=sum(np.logical_and(Peak_coordinate_array<=plotsx[i][-1], Peak_coordinate_array>plotsx[i][0]))
            tempxS=S_peak_time[last_dot_index:last_dot_index+number_of_dots]
            tempyS=S_peak_value[last_dot_index:last_dot_index+number_of_dots]
            tempxR=R_peak_time[last_dot_index:last_dot_index+number_of_dots]
            tempyR=R_peak_value[last_dot_index:last_dot_index+number_of_dots]
            last_dot_index+=number_of_dots
            plt.scatter(tempxS, tempyS)
            plt.scatter(tempxR, tempyR)
        
            plt.show()
        
        if do_i_need_to_ask:
            time.sleep(1)
            print("Last check. Is everything looks fine? No if no")
            time.sleep(1)
            ans=input()
            if ans == 'no' or ans == 'No':
                print("Chanel "+chanel_name+" is skiped. Do you want continue with other chanels? Yes or No")
                time.sleep(1)
                ans=input()
                if ans == 'yes' or ans == 'Yes':
                    pass
                else:
                    if len(list_of_results1)>0:
                        return list_of_results1, list_of_results2, list_of_results3
                    else:
                        return None
            time.sleep(1)
            print("Do you want to continue with automod? You can't go back if you choose this. Yes or no")
            time.sleep(1)
            ans=input()
            if ans == 'yes' or ans == 'Yes':
                do_i_need_to_ask=False
        
        list_of_results1=pd.concat([list_of_results1,pd.Series(Rarray).rename(chanel_name+' S value')], axis=1)
        list_of_results1=pd.concat([list_of_results1,pd.Series(Sarray).rename(chanel_name+' R value')], axis=1)
        list_of_results2=pd.concat([list_of_results2,pd.Series(Rarray-Sarray).rename(chanel_name+' RS distance')], axis=1)
        list_of_results3=pd.concat([list_of_results3,pd.Series(Peak_coordinate_array).rename(chanel_name+' peak time')], axis=1)
        
    
    return list_of_results1, list_of_results2, list_of_results3

def maincode(filename, header_value=6, begining_of_data_example=0, length_of_data_example=herz*60, truncator=0.005,  between_peak_min_distance_in_s=0.1, percent=0, 
           treshold =-20, prom=15, lenght_of_plot=60*herz, mint=0, hint=0, qs=10, qr=40):
    
    
    
    data=pd.read_csv(filename, header=header_value, sep=" ")
    do_i_need_to_ask=False
    
    if do_i_need_to_ask:
        print(data.head(10))
        time.sleep(1)
        print(" \nIs everything all right? Yes or No")
        time.sleep(1)
        ans=input()
        if ans == 'yes' or ans == 'Yes':
            pass
        else:
            print("Try to change header_value and check the csv file.")
            return None
    
    
    result = SRfinder(qr,qs,mint,hint,data, begining_of_data_example, length_of_data_example, truncator, between_peak_min_distance_in_s, percent, treshold, prom,  lenght_of_plot)
    
    
    
    if not (result==None):
        writer = pd.ExcelWriter(filename.split('.')[0]+"_output"+'.xlsx', engine='xlsxwriter')

        result[0].to_excel(writer, sheet_name='SRvalues')
        result[1].to_excel(writer, sheet_name='SRdistances')
        result[2].to_excel(writer, sheet_name='Rtimes')
        
        total_time = data.iat[-1,0]
        print(total_time)
        number_of_minutes=int(np.ceil(total_time/60))
        chanels=result[2].columns.values
        
        dicti={}
        for chanel_name in chanels:
            val=[]
            for i in range(number_of_minutes):
                temp=result[2][chanel_name].values
                #print(temp)
                number_of_beats=sum(np.logical_and(temp>=i*60, temp<(i+1)*60))
                val.append(number_of_beats)
            dicti[(chanel_name).split(' peak time')[0]+' beats per minute']=val
        newdf=pd.DataFrame(dicti)
        
        newdf.to_excel(writer,sheet_name='bpm')
        writer.save()
        
    return None


print(" \nstart")
maincode("L14P27_1001_kof-2021.10.05-10.17.45.725.out", treshold=-1.1, prom=0.1, hint=0, mint=0, qs=900, qr=850)

print(" \nstop")


KeyboardInterrupt: 